In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import os
from math import pi, sin, cos, sqrt
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
fms, tos, files = [], [], []
_base_  = '../../data/stanford.edu/facebook/'
_files_ = os.listdir(_base_)
for _file_ in _files_:
    if _file_.endswith('.edges'):
        _f_     = open(_base_ + _file_,'rt')
        _lines_ = _f_.read().split('\n')
        for _line_ in _lines_:
            _fmto_ = _line_.split(' ')
            if len(_fmto_) == 2:
                fms.append(_fmto_[0]+'_'+str(_file_)), tos.append(_fmto_[1]+'_'+str(_file_)), files.append(str(_file_))
df = pl.DataFrame({'fm':fms,'to':tos,'file':files}).filter(pl.col('file') == '0.edges')
high_degrees = set(['56_0.edges','67_0.edges','271_0.edges'])
df = df.filter(pl.col('fm').is_in(high_degrees) | pl.col('to').is_in(high_degrees))
rt.histogram(df, bin_by='fm', count_by='to', count_by_set=True)

In [ ]:
cd = rt.chordDiagram(df, [('fm','to')], link_style='bundled', skeleton_algorithm='hdbscan', link_opacity=0.1, link_size_min=2.0, link_size_max=4.0, x_ins=30, y_ins=30)
_svg_= cd._repr_svg_()
rt.tile([_svg_, cd.skeleton_svg])

In [ ]:

xmin,xmax,ymin,ymax = 10,220,10,220
cx,cy = xmin + (xmax - xmin)/2, ymin + (ymax - ymin)/2
r     = min((xmax-xmin)/2,(ymax-ymin)/2)
svg = [f'<svg x="0" y="0" width="{xmax+10}" height="{ymax+10}">']
svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="#a0a0a0" />')
svg.append(f'<circle cx="{cx}" cy="{cy}" r="2" fill="#ff0000" stroke="#ff0000" />')

r_div  = 10
hx_e   = r/r_div
hx_h   = sqrt(hx_e**2 - (hx_e/2)**2)
hx_p   = lambda x,y: f"M {x} {y} m {-hx_e} {0} l {hx_e/2} {hx_h} l {hx_e} {0} l {hx_e/2} {-hx_h} l {-hx_e/2} {-hx_h} l {-hx_e} {0} l {-hx_e/2} {hx_h}"
hx_pos = lambda x,y: [(x-hx_e,   y),         
                      (x-hx_e/2, y+hx_h), 
                      (x+hx_e/2, y+hx_h), 
                      (x+hx_e,   y), 
                      (x+hx_e/2, y-hx_h), 
                      (x-hx_e/2, y-hx_h), 
                      (x-hx_e,   y)]

for j in range(r_div):
    y_shift = hx_h if (j%2)==0 else 0.0
    for i in range(r_div):
        svg.append(f'<path d="{hx_p(cx-hx_e*1.5*(j),cy+y_shift-2*hx_h*(i))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{hx_p(cx-hx_e*1.5*(j),cy+y_shift+2*hx_h*(i))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{hx_p(cx+hx_e*1.5*(j),cy+y_shift-2*hx_h*(i))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{hx_p(cx+hx_e*1.5*(j),cy+y_shift+2*hx_h*(i))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
def pointsToPath(points):
    p = 'M '+str(points[0][0])+','+str(points[0][1])
    for i in range(1,len(points)):
        p += ' L '+str(points[i][0])+','+str(points[i][1])
    return p

svg = [f'<svg x="0" y="0" width="{xmax+10}" height="{ymax+10}">']
svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="#a0a0a0" />')
svg.append(f'<circle cx="{cx}" cy="{cy}" r="2" fill="#ff0000" stroke="#ff0000" />')

for j in range(r_div):
    y_shift = hx_h if (j%2)==0 else 0.0
    for i in range(r_div):
        svg.append(f'<path d="{pointsToPath(hx_pos(cx-hx_e*1.5*(j),cy+y_shift-2*hx_h*(i)))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{pointsToPath(hx_pos(cx-hx_e*1.5*(j),cy+y_shift+2*hx_h*(i)))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{pointsToPath(hx_pos(cx+hx_e*1.5*(j),cy+y_shift-2*hx_h*(i)))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
        svg.append(f'<path d="{pointsToPath(hx_pos(cx+hx_e*1.5*(j),cy+y_shift+2*hx_h*(i)))}" fill="none" stroke="#ff0000" stroke-width="0.4" />')

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = [f'<svg x="0" y="0" width="{xmax+10}" height="{ymax+10}">']
svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="#a0a0a0" />')
svg.append(f'<circle cx="{cx}" cy="{cy}" r="2" fill="#ff0000" stroke="#ff0000" />')
for j in range(r_div):
    y_shift = hx_h if (j%2)==0 else 0.0
    for i in range(r_div):
        for pnx in [-1, 1]:
            for pny in [-1, 1]:
                pts = hx_pos(cx+pnx*hx_e*1.5*(j),cy+y_shift+pny*2*hx_h*(i))
                for _edge_ in zip(pts,pts[1:]):
                    if rt.segmentLength((_edge_[0], (cx,cy))) < r*0.9 and rt.segmentLength((_edge_[1], (cx,cy))) < r*0.9:
                        svg.append(f'<path d="{pointsToPath(_edge_)}" fill="none" stroke="#ff0000" stroke-width="0.4" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
cd = rt.chordDiagram(df, [('fm','to')],
                     skeleton_algorithm='simple',
                     link_style='bundled', 
                     link_opacity=0.1, 
                     link_size_min=2.0, link_size_max=4.0, x_ins=30, y_ins=30)
_svg_= cd._repr_svg_()
rt.tile([_svg_, cd.skeleton_svg])

In [ ]:
cd = rt.chordDiagram(df, [('fm','to')],
                     skeleton_algorithm='hexagonal',
                     link_style='bundled', 
                     link_opacity=0.1, 
                     link_size_min=2.0, link_size_max=4.0, x_ins=30, y_ins=30)
_svg_= cd._repr_svg_()
rt.tile([_svg_, cd.skeleton_svg])

In [ ]:
params = {'df':df, 'relationships':[('fm','to')],'w':512,'h':512}
rt.table([rt.chordDiagram(**params),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hexagonal'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hdbscan'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='simple')], per_row=2)

In [ ]:
params = {'df':df, 'relationships':[('fm','to')],'w':512,'h':512, 'equal_size_nodes':True}
rt.table([rt.chordDiagram(**params),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hexagonal'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hdbscan'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='simple')], per_row=2)

In [ ]:
params = {'df':df, 'relationships':[('fm','to')],'w':512,'h':512, 'equal_size_nodes':True, 'link_color':'shade_fm_to', 'link_opacity':0.4}
rt.table([rt.chordDiagram(**params),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hexagonal'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='hdbscan'),
          rt.chordDiagram(**params, link_style='bundled', skeleton_algorithm='simple')], per_row=2)